# Install Bio Dependencies

In [ ]:
# Install bio dependencies
!pip install biopython dnacauldron dnachisel dnaweaver -q

# Import Bio Dependencies

In [ ]:
# Import bio dependencies
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Restriction import *
from Bio.SeqRecord import SeqRecord
import dnacauldron as dnc
import dnachisel as dch
import dnaweaver as dw

# Install Golden Gate dependencies

In [ ]:
# Install Golden Gate dependencies
!apt-get -qq install -y swig3.0
!ln /usr/bin/swig3.0 /usr/bin/swig
!pip install numberjack
!pip install goldenhinges
!pip install tatapov

# Import Golden Gate dependencies

In [ ]:
# Import Golden Gate dependencies
import tatapov
from goldenhinges import OverhangsSelector

# Define Golden Gate Site

In [ ]:
# Replace degenerate bases in Type IIS RS
def define_gg_site(enzyme):
  """
  Replace degenerate bases with random GC bases for T2S RS.
  """
  B = []
  for x in enzyme.elucidate().split("^")[0]:
    if "N" in x:
      B.append(dch.random_dna_sequence(length=1, gc_share=1, seed=0))
    else:
      B.append(x)
  return Seq("".join(B))

# Get Golden Gate Part

In [ ]:
# Get sequence between Golden Gate handles
def get_gg_part(enzyme, seq, overhang: int = 4):
  # Default overhang length is 4 nt.
  gg_part = enzyme.catalyse(seq)[1] + enzyme.catalyse(seq)[-1][:overhang]
  return gg_part

# Switch Golden Gate Handles

In [ ]:
# Switch Golden Gate handles
def switch_gg_handles(enzyme_1, enzyme_2, seq, overhang: int = 4):
  """
  Switch T2S enzyme handles for a given sequence.
  Default overhang length is 4 nt.
  """
  seq_ = get_gg_part(enzyme_1, seq, overhang)
  new_handle = Seq(define_gg_site(enzyme_2))
  new_seq = new_handle + seq_ + new_handle.reverse_complement()

  return Seq(new_seq)

# Generate Golden Gate Switching Primers

In [ ]:
# Generate primers to switch Golden Gate handles
def get_switching_handles(enzyme_1, enzyme_2, seq, overhang: int = 4):
  handle = Seq("GGCTAC") # Flanking bases for RE binding
  new_seq = switch_gg_handles(enzyme_1, enzyme_2, seq, overhang)
  new_seq_F = Seq(handle + enzyme_2.catalyse(new_seq)[0] + enzyme_2.catalyse(new_seq)[1][:20])
  new_seq_R = Seq(enzyme_2.catalyse(new_seq)[1][-20:] + enzyme_2.catalyse(new_seq)[-1] + handle).reverse_complement()
  return new_seq_F, new_seq_R

# Read DNA Sequence from FASTA

In [ ]:
# Read DNA sequence from FASTA file
def read_dna_from_fasta(f: str):
  with open(f, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
      dna_seq = record.seq
  return dna_seq

# Read DNA Sequence from GenBank

In [ ]:
# Read DNA sequence from GenBank file
def read_dna_from_genbank(f: str):
  with open(f, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
      dna_seq = record.seq
  return dna_seq

# Text to DNA

In [ ]:
# Text to DNA
def text_to_dna(text):
    # DNA mapping for 2-bit chunks
    mapping = {
        '00': 'A',
        '01': 'C',
        '10': 'G',
        '11': 'T'
    }

    dna_sequence = []

    for char in text:
        # Convert character to its ASCII value
        ascii_val = ord(char)
        # Convert ASCII value to an 8-bit binary string
        binary_str = format(ascii_val, '08b')  # e.g. 72 -> '01001000'

        # Split into pairs of bits (00, 01, 10, 11)
        for i in range(0, 8, 2):
            two_bits = binary_str[i:i+2]  # e.g. '01'
            dna_sequence.append(mapping[two_bits])  # Convert to A/C/G/T

    # Return a single continuous DNA string
    return "".join(dna_sequence)

# DNA to Text

In [ ]:
# DNA to text
def dna_to_text(dna_sequence):
    """
    Convert a DNA sequence back to text using the reverse
    mapping (DNA -> binary -> ASCII).
    """
    # Reverse mapping for single nucleotides
    reverse_mapping = {
        'A': '00',
        'C': '01',
        'G': '10',
        'T': '11'
    }

    # We'll decode every 4 nucleotides into 8 bits -> 1 character
    text_chars = []

    # Ensure the sequence length is a multiple of 4
    if len(dna_sequence) % 4 != 0:
        raise ValueError("DNA sequence length must be a multiple of 4.")

    for i in range(0, len(dna_sequence), 4):
        chunk = dna_sequence[i:i+4]  # e.g. 'CAGA'
        bits = ""

        # Convert each nucleotide back to its 2-bit string
        for nucleotide in chunk:
            if nucleotide not in reverse_mapping:
                raise ValueError(f"Invalid nucleotide '{nucleotide}' found.")
            bits += reverse_mapping[nucleotide]

        # Convert the 8-bit binary string to an integer ASCII code
        ascii_val = int(bits, 2)
        # Convert ASCII code to the corresponding character
        text_chars.append(chr(ascii_val))

    return "".join(text_chars)

# Get Protein Sequence

In [ ]:
# NCBI filler e-mail address
Entrez.email = "m02503275@gmail.com"

def get_protein_sequence(gene_symbol, retmax=1):
    """
    Fetches the first matching protein sequence for the given gene symbol.
    Returns a SeqRecord object containing the FASTA sequence.
    """
    # 1) Search for the gene symbol in the 'protein' database
    handle = Entrez.esearch(db="protein", term=gene_symbol, retmax=retmax)
    record = Entrez.read(handle)
    handle.close()

    if not record['IdList']:
        raise ValueError(f"No protein found for gene symbol '{gene_symbol}'")

    # 2) Fetch the FASTA sequence of the first matching protein
    protein_id = record['IdList'][0]
    handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
    seq_record = SeqIO.read(handle, "fasta")
    handle.close()

    return seq_record

# Extract Translation from DNA Sequence

In [ ]:
# Extract translation from DNA sequence
def extract_translation_from_dna(dna_seq):
    """
    Scans a DNA sequence from left to right to find:
      1) The first 'ATG' start codon (defining the reading frame).
      2) Continues in steps of 3 until the first stop codon.

    Returns the translated protein sequence as a string.

    If no start codon is found, returns an empty string.
    """
    start_codon = "ATG"
    stop_codons = {"TAA", "TAG", "TGA"}

    # 0) Conveert sequence to string and format to uppercase
    dna_seq = str(dna_seq).upper()

    # 1) Find the first occurrence of ATG
    start_idx = None
    for i in range(len(dna_seq) - 2):
        codon = dna_seq[i:i+3]
        if codon == start_codon:
            start_idx = i
            break

    # If no start codon, return empty
    if start_idx is None:
        return ""

    # 2) Starting from the found start codon, read in steps of 3
    codons = []
    for i in range(start_idx, len(dna_seq), 3):
        codon = dna_seq[i:i+3]
        if len(codon) < 3:
            break
        codons.append(codon)
        if codon in stop_codons:
            break  # Stop at the first in-frame stop codon

    # 3) Combine the codons into one string and add stop to end
    segment = "".join(codons) + "*"

    # 4) Translate using Biopython (stops at the first stop codon automatically)
    return Seq(segment).translate(to_stop=True)

# Find Closest Protein Match

In [ ]:
# Find closest protein match
def find_closest_protein_match(amino_acid_seq):
    """
    Performs a BLASTP search for the best match to a given amino acid sequence.

    Parameters
    ----------
    amino_acid_seq : str
        The protein sequence to query (single-letter amino acid codes).
    email : str
        Email address for NCBI (required by NCBI; include for usage tracking).

    Returns
    -------
    dict or None
        A dictionary with basic info on the top hit, including:
          - Accession
          - Title
          - E-value
          - % length match (coverage)
        or None if no significant hits are found.
    """
    result_handle = NCBIWWW.qblast(
        program="blastp",
        database="nr",
        sequence=amino_acid_seq,
        format_type="XML",
        expect=0.001,       # E-value cutoff, adjust as needed
        hitlist_size=1     # Only fetch the top hit for simplicity
    )

    blast_records = NCBIXML.parse(result_handle)

    for blast_record in blast_records:
        if not blast_record.alignments:
            # No hits in this BLAST record
            continue

        top_alignment = blast_record.alignments[0]
        top_hsp = top_alignment.hsps[0]  # High-scoring pair

        # Calculate % length match (coverage):
        # how much of the query is covered by the alignment
        pct_length_match = 100.0 * top_hsp.align_length / len(amino_acid_seq)

        return {
            "accession": top_alignment.accession,
            "title": top_alignment.title,
            "e_value": top_hsp.expect,
            "percent_length_match": pct_length_match
        }

    # If we reach here, there were no hits at all
    return None

# Codon Optimize Protein Sequence

In [ ]:
# Codon optimize protein sequence
def codon_optimize(protein_seq, avoid_enzymes, species, stop=False, min_gc: float = 0.4, max_gc: float = 0.6, gc_window: int = 50):
  """
  Codon optimize a protein sequence.
  Avoid specified RE sites.
  Opt-in STOP codon.
  """
  # Coding sequence by back-translation
  cds = dch.reverse_translate(protein_seq)

  # Pattern for enzymes to avoid
  avoid_pattern = dch.SequencePattern("|".join([x.site for x in RestrictionBatch(avoid_enzymes)]))
  print(avoid_pattern)

  # Optimize DNA sequence
  problem = dch.DnaOptimizationProblem(
    sequence=cds,
    constraints=[
        dch.AvoidPattern(avoid_pattern),
        dch.EnforceGCContent(mini=min_gc, maxi=max_gc, window=gc_window),
        dch.EnforceTranslation(),
    ],
    objectives=[dch.CodonOptimize(species=species)]
  )
  problem.resolve_constraints()
  problem.optimize()

  # Optimize stop codon
  if not stop:
    return problem.sequence

  stop_codon = dch.reverse_translate("*")
  stop_problem = dch.DnaOptimizationProblem(
      sequence = stop_codon,
      objectives=[dch.CodonOptimize(species=species)]
  )
  stop_problem.resolve_constraints()
  stop_problem.optimize()
  stop_seq = stop_problem.sequence

  return problem.sequence + stop_seq

# Vector Overhangs

In [ ]:
# Vector overhangs
# 1_pU6_sgRNA(mRFP)
# 2_pCMV_2A-eGFP_bGH(mRFP)
# 3_pPGK_hGH
# 4_EC_K12 expression cassettes
vector_overhangs = {1: ["CACC", "GTTT"],
                    2: ["CACC", "GGAT"],
                    3: ["ACC", "AGC"],
                    4: ["AATG", "GCTT"]}

# Avoid Restriction Sites

In [ ]:
# Avoid restriction sites
avoid_sites_1 = ["BsaI", "BsmBI", "EcoRV"]
avoid_sites_2 = ["BsaI", "BsmBI", "EcoRV"]
avoid_sites_3 = ["SapI", "EcoRV"]
avoid_sites_4 = ["BsaI", "BsmBI"]

# Add Golden Gate Overhangs

In [ ]:
# Add Golden Gate overhangs
def add_golden_gate_overhangs(seq, enzyme, vector_id):
  """
  Add Golden Gate Overhangs.
  """
  seq = Seq(seq)

  # Vector overhangs
  left_overhang, right_overhang = vector_overhangs.get(vector_id)

  # Golden Gate sites
  gg_site = define_gg_site(enzyme)
  left_site, right_site = gg_site + left_overhang, right_overhang + gg_site.reverse_complement()

  return Seq(left_site + seq + right_site)

# Get Maximum Set of Golden Gate Overhangs

In [ ]:
# Get maximum set of Golden Gate overhangs
# Using graph approach to find maximum clique
import networkx as nx
import itertools
from networkx.algorithms.approximation import clique

def generate_all_sequences(length):
    """Generate all possible DNA sequences of a given length."""
    bases = 'ATCG'
    return [''.join(seq) for seq in itertools.product(bases, repeat=length)]

def hamming_distance(seq1, seq2):
    """Compute the Hamming distance between two sequences."""
    return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))

def get_max_gg_overhangs(overhang_length: int = 4, min_dist: int = 2):
    # Step 1: Generate all possible DNA sequences of the target length.
    sequences = generate_all_sequences(overhang_length)

    # Step 2: Create a graph where sequences are nodes,
    # and add edges between nodes if their Hamming distance meets the minimum.
    G = nx.Graph()
    G.add_nodes_from(sequences)

    for seq1, seq2 in itertools.combinations(sequences, 2):
        if hamming_distance(seq1, seq2) >= min_dist:
            G.add_edge(seq1, seq2)

    # Step 3: Use NetworkX's approximation algorithm to find a large clique.
    max_clq = clique.max_clique(G)

    # Step 4: Return the list of sequences in the (approximate) maximum clique.
    return list(max_clq)

# Get Maximum Set of Golden Gate Overhangs (DAD)

In [ ]:
# Get maximum set of Golden Gate DAD overhangs
def get_max_gg_dad_overhangs(enzyme_id, temp_id: str, dist: int = 2):
  """
  Get maximum set of Golden Gate overhangs.
  Based on Tatapov DAD (enzyme, temperature).
  Set base difference (default = 2).
  """
  annealing_data = tatapov.annealing_data[temp_id][enzyme_id]

  self_annealings = tatapov.relative_self_annealings(annealing_data)
  weak_self_annealing_overhangs = [
      overhang
      for overhang, self_annealing in self_annealings.items()
      if self_annealing < 0.05
  ]

  cross_annealings = tatapov.cross_annealings(annealing_data)
  high_cross_annealing_pairs = [
      overhang_pair
      for overhang_pair, cross_annealing in cross_annealings.items()
      if cross_annealing > 0.005
  ]

  selector = OverhangsSelector(
      difference=dist,
      forbidden_overhangs=weak_self_annealing_overhangs,
      forbidden_pairs=high_cross_annealing_pairs
  )

  overhangs = selector.generate_overhangs_set()

  return overhangs

# Create gene name

In [ ]:
# Create gene name
def create_gene_name(gene_symbol, species: str, vector_id: int, chemistry: str):
  """
  Create gene name based on symbol, species and vector_id.
  """
  prefix = "".join([str(vector_id), chemistry])
  gene_name = "_".join([prefix, species, gene_symbol])

  return gene_name